# SuppressImplicit
* Zero out any items the user has already seen and rescale the probablity distribution

In [ ]:
function get_seen_probabilities(alpha::String, content::String)
    df = read_alpha_impl(alpha, "training", content)
    p_seen = zeros(Float32, num_users(), Threads.nthreads())
    Threads.@threads for i = 1:length(df.user)
        p_seen[df.user[i], Threads.threadid()] += df.rating[i]
    end
    vec(sum(p_seen, dims = 2))
end

function read_alpha(alpha::String, split::String, content::String)
    @assert !occursin("training", split)
    ϵ = sqrt(eps(Float32))
    df = read_alpha_impl(alpha, split, content)
    if alpha in implicit_raw_alphas || alpha in ptw_raw_alphas
        # zero out any items the user has already seen
        p_seen = get_seen_probabilities(alpha, content)
        Threads.@threads for i = 1:length(df.rating)
            if  1 - p_seen[df.user[i]] > ϵ
                df.rating[i] /= 1 - p_seen[df.user[i]]
            end
        end
    end
    df
end;